In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
news = pd.read_csv('/content/drive/MyDrive/datasets/trainset.txt',sep='\t', names = ['CLASS','TITLE','DATE','BODY'])
news.head()

,CLASS,TITLE,DATE,BODY
0,1,JAPAN FIRM PLANS TO SELL U.S. FARMLAND TO JAP...,"MORIOKA, Japan, March 12 -",A Japanese real estate company said it will ...
1,-1,NORTH BH SETS ONE-FOR-FIVE OFFER FOR NORGOLD ...,"MELBOURNE, March 12 -",North Broken Hill Holdings Ltd & lt;NBHA.ME ...
2,-1,OUTOKUMPU IN COPPER DEAL WITH IBERICA DEL COBRE,"HELSINKI, June 26 -",Finland's state-owned mining company Outokum...
3,1,ROTTERDAM GRAIN HANDLER SAYS PORT BALANCE ROSE,"ROTTERDAM, April 13 -","Graan Elevator Mij, GEM, said its balance in..."
4,1,U.S. SENATE PANEL VOTES TO LIMIT COUNTY LOAN ...,(NO DATE),(NO TEXT)


In [ ]:
import nltk
nltk.download('all')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    |   Package bcp47 is already up-to-dat

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

def preprocess(text):
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))  # Corrected set name to 'english'
    tokens = [token for token in tokens if token.lower() not in stop_words]
    tokens = [token for token in tokens if len(token)>=3]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token.lower()) for token in tokens]
    return tokens
#preprocess("a japanese real estate company")
#process(news['BODY'][0])
news['BODY2'] = news['BODY'].apply(preprocess)
news

,CLASS,TITLE,DATE,BODY,BODY2
0,1,JAPAN FIRM PLANS TO SELL U.S. FARMLAND TO JAP...,"MORIOKA, Japan, March 12 -",A Japanese real estate company said it will ...,"[japanese, real, estate, company, said, launch..."
1,-1,NORTH BH SETS ONE-FOR-FIVE OFFER FOR NORGOLD ...,"MELBOURNE, March 12 -",North Broken Hill Holdings Ltd & lt;NBHA.ME ...,"[north, broken, hill, holding, ltd, nbha.me, n..."
2,-1,OUTOKUMPU IN COPPER DEAL WITH IBERICA DEL COBRE,"HELSINKI, June 26 -",Finland's state-owned mining company Outokum...,"[finland, state-owned, mining, company, outoku..."
3,1,ROTTERDAM GRAIN HANDLER SAYS PORT BALANCE ROSE,"ROTTERDAM, April 13 -","Graan Elevator Mij, GEM, said its balance in...","[graan, elevator, mij, gem, said, balance, por..."
4,1,U.S. SENATE PANEL VOTES TO LIMIT COUNTY LOAN ...,(NO DATE),(NO TEXT),[text]
...,...,...,...,...,...
195,-1,CITY RESOURCES UNIT ACQUIRES PHILIPPINE MINE ...,"HONG KONG, April 21 -","& lt;City Resources (Asia) Ltd > , a locally...","[city, resource, asia, ltd, locally, listed, u..."
196,1,"TAIWAN TO TENDER FOR 27,000 TONNES U.S. SOYBEANS","TAIPEI, March 11 -",The joint committee of Taiwan's soybean impo...,"[joint, committee, taiwan, soybean, importer, ..."
197,-1,CAROLIN MINES & lt;CRLNF > HAS NEW ASSAY RESULTS,"Vancouver, British Columbia, March 9 -",Carolin Mines Ltd said recent assays of tail...,"[carolin, mine, ltd, said, recent, assay, tail..."
198,-1,TWO JAPANESE STEELMAKERS' CAPITAL SPENDING FALLS,"TOKYO, March 27 -",Kawasaki Steel Corp & lt;KAWS.T > said its p...,"[kawasaki, steel, corp, kaws.t, said, parent, ..."


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Convert the list of tokens to a single string for each document
news['BODY_str'] = news['BODY2'].apply(lambda x: ' '.join(x))

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(news['BODY_str'])
X.shape

(200, 4391)

In [ ]:
feature_names = vectorizer.get_feature_names_out()
feature_names

array(['00', '000', '008', ..., 'zone', 'zurich', 'zuyuan'], dtype=object)

In [ ]:
from sklearn.feature_selection import mutual_info_classif
import numpy as np


info_gain = mutual_info_classif(X,news['CLASS'])

feature_ranked = np.argsort(info_gain)[::-1]


for i in feature_ranked[:10]:
  print(f"Feature :{feature_names[i]}, Information Gain : {info_gain[i]}")

Feature :company, Information Gain : 0.15508340001179602
Feature :mine, Information Gain : 0.15409341691480502
Feature :gold, Information Gain : 0.15409341691480494
Feature :wheat, Information Gain : 0.12242482379159635
Feature :inc, Information Gain : 0.11632770527573652
Feature :mining, Information Gain : 0.11181138665716192
Feature :copper, Information Gain : 0.1118113866571619
Feature :agriculture, Information Gain : 0.10947658698244675
Feature :ltd, Information Gain : 0.10168306585999437
Feature :ounce, Information Gain : 0.09851327083554395
